In [24]:
import numpy as np
import open3d as o3d
from tqdm import tqdm

In [25]:
datas = o3d.io.read_point_cloud('Line11.ply')
xyz = np.asarray(datas.points)

In [26]:
xyz = xyz[np.lexsort(xyz.T)]

In [27]:
Nlrp = 1000
LRP = np.average(xyz[:Nlrp,2])

In [28]:
Thseeds = 0.1

In [29]:
seed = xyz[xyz[:,2] < Thseeds + LRP ]

In [30]:
def estimate_parameters(pts):
        # 最小二乘法估算平面模型
        # 只有三个点时，可以直接计算
        num = pts.shape[0]
        if num == 3:
            c = np.mean(pts,axis=0)
            l1 = pts[1]-pts[0]
            l2 = pts[2]-pts[0]
            n = np.cross(l1,l2)
            scale = [n[i]**2 for i in range(n.shape[0])]
            #print(scale)
            n = n/np.sqrt(np.sum(scale))
        else:
            _,_,c,n = SVD(pts)

        params = np.hstack((c.reshape(1,-1),n.reshape(1,-1)))[0,:]
        return params

In [31]:
seed_cp = seed
for i in range(10):    
    params = estimate_parameters(pts=seed_cp)
    h = (xyz[:,0]-params[0]) * params[3] + (xyz[:,1]-params[1])* params[4]+((xyz[:,2]-params[2])* params[5])/(np.sqrt(params[3]**2+params[4]**2+params[5]**2))
    seed_cp = xyz[h < 0.2]

In [32]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(seed_cp)
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
